## 构造 FaissDB

### 从 Embeddings 构建文档

In [1]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import MarkMeta, FaissDB

# 导入文档
local = MarkMeta(dir="./", chunk_size=550)
local(verbose=True)

[INFO] 已成功加载文件 __DOCS__/./001.md ，其中包含 10 个片段。
[INFO] 已成功加载文件 __DOCS__/./设计理念.md ，其中包含 14 个片段。
[INFO] 已成功加载文件 __DOCS__/./推理模式.md ，其中包含 19 个片段。


[Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="# illufly 设计理念介绍...", meta=['source', 'id']),
 Document(text="一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：...", meta=['source', 'id']),
 Document(text="现在，要求你准确记住这些类的名字以及该从哪里引用，就开始慢慢形成挑战了。...", meta=['source', 'id']),
 Document(text="'role': 'user',...", meta=['source', 'id']),
 Document(text="# 常见的智能体推理模式和 illufly 的实现...", meta

In [2]:
# 文档嵌入
emb = TextEmbeddings()
emb(local.documents, verbose=True)

[Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="# illufly 设计理念介绍...", meta=['source', 'id', 'embeddings']),
 Document(text="一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：...", meta=['source', 'id', 'embeddings']),
 Document(text="现在，要求你准确记住

In [3]:
# 文档入库
faiss = FaissDB(emb, train=False)

# 查询
faiss("怎么做披萨？", verbose=True)

[INFO] 查询到5条结果


[Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance'])]

### 从 VectorDB 添加文本

In [5]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import FaissDB

# 文档嵌入
db = FaissDB(TextEmbeddings())
db.load_text("普鸿是一家做数字消防业务的公司")
db.load_text("幻蝶是一家AI技术公司")

# 查询
db("普鸿是啥？", top_k=5, verbose=True)

[INFO] 查询到2条结果


[Document(text="普鸿是一家做数字消防业务的公司", meta=['raw_meta', 'id', 'source', 'embeddings', 'distance']),
 Document(text="幻蝶是一家AI技术公司", meta=['raw_meta', 'id', 'source', 'embeddings', 'distance'])]

### 从 VectorDB 添加本地文件

In [6]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import FaissDB

# 文档嵌入
db = FaissDB(TextEmbeddings())
db.load("./", chunk_size=550)
# 查询
db("披萨要烤几分钟？", top_k=1)

[Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance'])]